# Localiza desafio 2

Antes de apresentar as análises para a liderança, você decide se reunir com seu colega
de trabalho para discutirem os resultados e se estruturarem para a apresentação. No entanto,
para isso, é preciso identificar um horário na agenda que seja factível para ambos.

**Dadas as agendas de duas pessoas e seus respectivos horários de trabalho, escreva
um algoritmo que retorna todos os horários nos quais ambas as pessoas estariam disponíveis
para uma reunião de t minutos.**

As agendas serão compostas por uma lista de n compromissos previamente marcados.
Esses compromissos, por sua vez, serão representados por uma lista ou tuple de duas strings,
no qual o primeiro elemento representa o horário de início do compromisso e o último elemento
o horário de término. A jornada de trabalho terá o mesmo formato de um compromisso, no qual
o primeiro elemento representa o horário de início do turno e o último elemento o horário final da
jornada. O tempo da reunião, em minutos, será um inteiro. Por fim, espera-se que o resultado
esteja no mesmo formato das agendas, nos quais os “compromissos”, nesse caso,
correspondem aos horários disponíveis para a reunião.

**Exemplo:** Considere o seguinte cenário:
1. Agenda da pessoa A: [['9:00', '10:30'], ['12:00', '13:00'], ['16:00', '18:00']]
2. Horário de Trabalho da pessoa A: ['9:00', '20:00']
3. Agenda da pessoa B: [['10:00', '11:30'], ['12:30', '14:30'], ['14:30', '15:00'], ['16:00',
'17:00']]
4. Horário de Trabalho da pessoa B: ['10:00', '18:30']

Os horários que possibilitam uma reunião de 30 minutos entre a pessoa A e pessoa B são:
[['11:30', '12:00'], ['15:00', '16:00'], ['18:00', '18:30']]

Nesse primeiro pedaço de código estou apenas inputando os horários e jornadas de trabalho descritas no enunciado:

In [4]:
schedules = {'A': [],
             'B': []}

journey = {'A': [],
           'B': []}

schedules['A'].append(['9:00', '10:30'])
schedules['A'].append(['12:00', '13:00'])
schedules['A'].append(['16:00', '18:00'])

journey['A'].append(['9:00', '20:00'])

schedules['B'].append(['10:00', '11:30'])
schedules['B'].append(['12:30', '14:30'])
schedules['B'].append(['14:30', '15:00'])
schedules['B'].append(['16:00', '17:00'])

journey['B'].append(['10:00', '18:30'])


No segundo pedaço de código estou declarando duas funções auxiliares, elas me ajudarão a navegar entre dois formatos de horário:
 - horários em HH:MM, "user-friendly" para o usuário ler melhor (ou inputar em versões mais sofisticadas em que o usuário adiciona outros horários)
 - número de minutos, de uso interno, que permite fazer cálculos e comparações matemáticas

In [1]:
def timeToMinutes(t):
  # This function returns number of minutes of a time object t written as HH:MM.
  hour = t.split(':')[0]
  minute = t.split(':')[1]
  return int(hour)*60 + int(minute)

def minutesToTime(t):
  # This function returns a time written as HH:MM from a t number of minutes.
  hour = int(t/60)
  hour = '0'+str(hour)
  hour = hour[-2:]
  minute = int(t)%60
  minute = '0'+str(minute)
  minute = minute[-2:]
  return hour+':'+minute


Nesse terceiro pedaço definirei outra função que vai me auxiliar na função principal, ela define em que horários do dia irei procurar por disponibilidade na agenda, dado o horário de trabalho de cada agente. Basicamente define uma "jornada comum" entre as duas jornadas de trabalho.

In [2]:
def commonJourney():
  #this function defines where to look for available time slots depending on each person's working journey, 
  #In other words we are looking for timeframes when all people are working
  
  starting_minutes = {}
  ending_minutes = {}

  for person in journey:
    starting_minutes[person] = timeToMinutes(journey[person][0][0])
    ending_minutes[person] = timeToMinutes(journey[person][0][1])
  # common journey begins on arrival time of second person in work
  begin = max(starting_minutes['A'],starting_minutes['B'])
  # common journey ends on departure time of first person in work
  finish = min(ending_minutes['A'],ending_minutes['B'])
  return begin, finish

Por fim, nesse último pedaço defino e testo a função principal solicitada no enunciado:

In [5]:

def matchSchedules(t):
  # This function returns all possible meetings of t minutes between 2 agendas with other schedules 

  # uses common journey to know which times to look
  loop_start, loop_stop = commonJourney()

  #this list will store all available times found in the loop
  available_starttime = [] 

  #this second step searches for a time interval (from loop_start to loop_start + t) is free in both agendas in this loop
  #this has to be done by matching this interval with every appointment of every person's schedule (that's why there's 2 for loops inside)
  while loop_start < loop_stop:
    #this auxiliary variable will help breaking out of both fors at the same test and telling a time is unavailable
    doublebreak = False
    print(f'checando disponibilidade às {minutesToTime(loop_start)}')

    for person in schedules:
      for appointment in schedules[person]:
        #checking if starting time of interval is in the middle of an appointment
        if timeToMinutes(appointment[0]) <= loop_start and timeToMinutes(appointment[1]) > loop_start:
          print('indisponível')
          doublebreak = True
          break 
        #checking if end time of interval is in the middle of an appointment
        elif timeToMinutes(appointment[0]) < loop_start + t and timeToMinutes(appointment[1]) >= loop_start + t:
          print('indisponível')
          doublebreak = True
          break
      if doublebreak:
        break
    
    #an interval will get here only if does not conflict to any appointment of any person's agenda
    if not doublebreak:
      print('disponível')
      available_starttime.append(loop_start)  
    loop_start = loop_start + t
  
  #last step will give the final result
  #piece code inside "if" will execute only if no appointment of size t was possible between both agendas
  if len(available_starttime) == 0:
    print(f'\n Não há horários de {t} minutos possíveis')
  else:
    print(f'\n Abaixo os horários de {t} minutos possíveis:')
    #This for will convert starting minutes into readable HH:MM - HH:MM intervals
    available_schedules = []
    for schedule in available_starttime:
      available_schedules.append([minutesToTime(schedule),minutesToTime(schedule+t)])
    print(available_schedules)
    
matchSchedules(30)

checando disponibilidade às 10:00
indisponível
checando disponibilidade às 10:30
indisponível
checando disponibilidade às 11:00
indisponível
checando disponibilidade às 11:30
disponível
checando disponibilidade às 12:00
indisponível
checando disponibilidade às 12:30
indisponível
checando disponibilidade às 13:00
indisponível
checando disponibilidade às 13:30
indisponível
checando disponibilidade às 14:00
indisponível
checando disponibilidade às 14:30
indisponível
checando disponibilidade às 15:00
disponível
checando disponibilidade às 15:30
disponível
checando disponibilidade às 16:00
indisponível
checando disponibilidade às 16:30
indisponível
checando disponibilidade às 17:00
indisponível
checando disponibilidade às 17:30
indisponível
checando disponibilidade às 18:00
disponível

 Abaixo os horários de 30 minutos possíveis:
[['11:30', '12:00'], ['15:00', '15:30'], ['15:30', '16:00'], ['18:00', '18:30']]
